In [ ]:
import matplotlib.pyplot as plt
import cv2
import importlib

import scipy

from pathlib import Path

import numpy as np

import h5py
import math


import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import holoviews as hv
hv.extension('bokeh')

import platform

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../../..")


from pathlib import Path

import cv2

import json

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook
import iqplot
import bokeh.io
bokeh.io.output_notebook()

import Utilities.Ballpushing_utils
from Utilities.Utils import *
from Utilities.Processing import *
from Utilities.Ballpushing_utils import *

# Get a list of the directories containing the tracking data

In [ ]:
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)

In [ ]:
Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "tnt_fine" in minfolder and "tracked" in minfolder:
        Folders.append(folder)

Folders

# Generating the dataset and adding the labels

In [ ]:
Dataset = generate_dataset(Folders, Events="interactions")

Dataset.reset_index()


In [ ]:

Dataset.head()

In [ ]:
# Remove unnecessary columns, Period, FeedingState, Orientation, Light and Crossing, and yball

Dataset = Dataset.drop(columns=["Period", "FeedingState", "Orientation", "Light", "Crossing", "yball"])

In [ ]:
regions_map = pd.read_csv("/mnt/labserver/DURRIEU_Matthias/Experimental_data/Region_map_231215.csv")


In [ ]:

regions_map.head()

In [ ]:
# Rename the "Codename" column to "Genotype"

regions_map = regions_map.rename(columns={"Codename": "Genotype"})

In [ ]:
# Add columns to the main dataset based on the regions map

Dataset = pd.merge(Dataset, regions_map, on="Genotype")

#Dataset.head()

# Number of events

In [ ]:
# For each Fly, compute how many unique values of Event that are not None there are

# Group the data by Fly and Event
GroupedData = Dataset.groupby(["Fly", "Nickname", "Simplified region"]).nunique(["Event"]).reset_index()

# Count the number of unique values of Event for each Fly
# Counts = GroupedData["Event"].nunique()
#GroupedData.head()

In [ ]:
# Redo the same plot but one plot per region

h_NumbEvents_bp = (
    hv.BoxWhisker(
        data=GroupedData,
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_NumbEvents_sc = (
    hv.Scatter(
        data=GroupedData,
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_NumbEvents = (h_NumbEvents_bp * h_NumbEvents_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Number of Events",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)


In [ ]:

hvplot_NumbEvents

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_NumbEvents,
    Savepath / "NumberEvents.html",
)

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

import pickle

# Save the plot
with open(Savepath / "hvplot_NumbEvents.pkl", "wb") as f:
    pickle.dump(hvplot_NumbEvents, f)

# Final event

In [ ]:
end_threshold = 10

# Group by 'Fly' column and find the maximum 'yball_relative' for each group
max_positions = Dataset.groupby("Fly")["yball_relative"].max()

# Initialize an empty DataFrame to store the results
result = []

# For each Fly, find the first event where 'yball_relative' is equal to max_position
for fly, max_position in max_positions.items():
    print(f"{fly} has a max position of {max_position}")
    fly_data = Dataset[Dataset["Fly"] == fly]
    fly_data = fly_data.sort_values("Event")
    event = (
        fly_data[fly_data["yball_relative"] >= max_position - end_threshold]["Event"]
        .drop_duplicates()
        .iloc[0]
    )
    result.append({"Fly": fly, "Event": event})

# Convert the result to a DataFrame
result_df = pd.DataFrame(result)

# Merge the result_df with the original Dataset
Dataset = pd.merge(Dataset, result_df, on=["Fly", "Event"], how="left")

# Create the 'IsFinal' column, which is True if 'Event' is in result_df and False otherwise
Dataset["IsFinal"] = False
Dataset.loc[Dataset["Event"].isin(result_df["Event"]), "IsFinal"] = True

#Dataset.head()

In [ ]:
Dataset.groupby("Fly")["yball_relative"].max()

Dataset["max_position"] = Dataset.groupby("Fly")["yball_relative"].transform("max")

end_threshold = 10
Dataset["IsFinal"] = Dataset["yball_relative"] >= (
    Dataset["max_position"] - end_threshold
)

# Dataset.head()
# Drop rows where 'Event' is NaN
# Dataset = Dataset.dropna(subset=['Event'])

# Now extract the event number from the 'Event' column
# Dataset['EventNumber'] = Dataset['Event'].str.extract('(\d+)').astype(int)

# Filter rows where 'IsFinal' is True
final_events_trimmed = Dataset[Dataset["IsFinal"] == True]

# final_events_trimmed = final_events_trimmed.dropna(subset=['Event'])
final_events_trimmed.head()

first_final = final_events_trimmed.groupby(["Fly"])["Event"].min()
first_final
# Create a new column 'FinalEvent' in the original DataFrame
Dataset = Dataset.merge(
    first_final.rename("FinalEvent"), left_on="Fly", right_index=True, how="left"
)


In [ ]:

# Create a new DataFrame with one row per fly
data_per_fly = Dataset.drop_duplicates(subset="Fly")
# Dataset.head()

In [ ]:
# Redo the same plot but one plot per region

h_FinalEvents_bp = (
    hv.BoxWhisker(
        data=data_per_fly,
        vdims="FinalEvent",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_FinalEvents_sc = (
    hv.Scatter(
        data=data_per_fly,
        vdims="FinalEvent",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_FinalEvents = (h_FinalEvents_bp * h_FinalEvents_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Event at which the fly reaches the end of the corridor",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)


In [ ]:

hvplot_FinalEvents

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_FinalEvents,
    Savepath / "FinalEvents.html",
)

In [ ]:
# Save the plot
with open(Savepath / "hvplot_FinalEvents.pkl", "wb") as f:
    pickle.dump(hvplot_FinalEvents, f)

# Time of final event

In [ ]:
# Create a new DataFrame with rows where 'Event' equals 'FinalEvent'
final_event_rows = Dataset[Dataset["Event"] == Dataset["FinalEvent"]]

# Drop duplicates based on 'Fly' to keep only the first occurrence of each fly
final_event_rows = final_event_rows.drop_duplicates(subset="Fly")

In [ ]:
# Holoviews version

h_TimeLastEvents_bp = (
    hv.BoxWhisker(
        data=final_event_rows,
        vdims="time",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_TimeLastEvents_sc = (
    hv.Scatter(
        data=final_event_rows,
        vdims="time",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_TimeLastEvents = (h_TimeLastEvents_bp * h_TimeLastEvents_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Time(s)",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_TimeLastEvents.pkl", "wb") as f:
    pickle.dump(hvplot_TimeLastEvents, f)

In [ ]:

hvplot_TimeLastEvents

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_TimeLastEvents,
    Savepath / "TimeLastEvents.html",
)

# Significant events

In [ ]:
def check_yball_variation(event_df, threshold=10):
    yball_segment = event_df["yball_smooth"]
    variation = yball_segment.max() - yball_segment.min()
    return variation > threshold


# Apply the function and reset the index
significant_events = (
    Dataset.groupby(["Fly", "Event"]).apply(check_yball_variation).reset_index()
)

# Rename the 0 column to 'SignificantEvent'
significant_events.rename(columns={0: "SignificantEvent"}, inplace=True)

# Merge the significant_events DataFrame with the original Dataset
Dataset = pd.merge(Dataset, significant_events, on=["Fly", "Event"], how="left")

#Dataset.head()

In [ ]:
# Count the number of unique events that are significant for each 'Fly' and 'Genotype'
unique_significant_events = Dataset[Dataset['SignificantEvent'] == True].groupby(['Fly', 'Nickname', 'Simplified region'])['Event'].nunique()

#print(unique_significant_events)


In [ ]:
h_unique_significant_events_bp = (
    hv.BoxWhisker(
        data=unique_significant_events.reset_index(),
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_unique_significant_events_sc = (
    hv.Scatter(
        data=unique_significant_events.reset_index(),
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_unique_significant_events = (
    h_unique_significant_events_bp * h_unique_significant_events_sc
).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Number of significant events",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_unique_significant_events.pkl", "wb") as f:
    pickle.dump(hvplot_unique_significant_events, f)

In [ ]:

hvplot_unique_significant_events

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_unique_significant_events,
    Savepath / "NumberSignificantEvents.html",
)

# Time of first significant event

In [ ]:
# Filter the DataFrame to include only the significant events
significant_events = Dataset[Dataset['SignificantEvent'] == True]

# For each 'Fly', find the first significant event and get its time
first_significant_event_times = significant_events.groupby('Fly')['time'].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]

first_significant_events.head()

# For each 'Fly' and 'Genotype', find the first significant event and get its time
first_significant_event_times = significant_events.groupby(['Fly', 'Nickname', "Simplified region"])['time'].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]


In [ ]:
# Holoviews version

h_TimeSignifEvents_bp = (
    hv.BoxWhisker(
        data=first_significant_events,
        vdims="time",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_TimeSignifEvents_sc = (
    hv.Scatter(
        data=first_significant_events,
        vdims="time",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_TimeSignifEvents = (h_TimeSignifEvents_bp * h_TimeSignifEvents_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Time(s)",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="Time of first significant event",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_TimeSignifEvents.pkl", "wb") as f:
    pickle.dump(hvplot_TimeSignifEvents, f)

In [ ]:

hvplot_TimeSignifEvents

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_TimeSignifEvents,
    Savepath / "TimeSignifEvent.html",
)

# Number of first significant event

In [ ]:
# Filter the DataFrame to include only the significant events
significant_events = Dataset[Dataset["SignificantEvent"] == True]

# For each 'Fly', find the first significant event and get its time
first_significant_event_times = significant_events.groupby("Fly")["time"].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]

# first_significant_events["EventNumber"] = first_significant_events.loc[:,"Event"].str.extract('(\d+)').astype(int)

#first_significant_events.head()

In [ ]:
h_first_significant_event_numbers_bp = (
    hv.BoxWhisker(
        data=first_significant_events.reset_index(),
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_first_significant_event_numbers_sc = (
    hv.Scatter(
        data=first_significant_events.reset_index(),
        vdims="Event",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_first_significant_event_numbers = (
    h_first_significant_event_numbers_bp * h_first_significant_event_numbers_sc
).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Number of first significant event",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_first_significant_event_numbers.pkl", "wb") as f:
    pickle.dump(hvplot_first_significant_event_numbers, f)

In [ ]:

hvplot_first_significant_event_numbers

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_first_significant_event_numbers,
    Savepath / "FirstSignifEvent.html",
)

# Pauses

In [ ]:
import numpy as np
Dataset['Event'] = Dataset['Event'].replace('None', np.nan)


In [ ]:
Dataset['IsEvent'] = Dataset['Event'].notna()


In [ ]:
Dataset['TimeSinceLastEvent'] = Dataset.groupby('Fly').apply(lambda x: x['time'] - x.loc[x['IsEvent'], 'time'].shift()).reset_index(level=0, drop=True)


In [ ]:
## For each 'Fly', calculate the time difference between consecutive events
Dataset['TimeSinceLastEvent'] = Dataset[Dataset['IsEvent']].groupby('Fly')['time'].diff()

# Calculate the average time between events for each 'Fly' and 'Genotype'
average_time_between_events = Dataset.groupby(['Fly', 'Nickname', 'Simplified region'])['TimeSinceLastEvent'].mean().reset_index()


In [ ]:
h_pauses_bp = (
    hv.BoxWhisker(
        data=average_time_between_events.reset_index(),
        vdims="TimeSinceLastEvent",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_pauses_sc = (
    hv.Scatter(
        data=average_time_between_events.reset_index(),
        vdims="TimeSinceLastEvent",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_pauses = (h_pauses_bp * h_pauses_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Average time between events (s)",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_pauses.pkl", "wb") as f:
    pickle.dump(hvplot_pauses, f)

In [ ]:

hvplot_pauses

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_pauses,
    Savepath / "AvgPauses.html",
)

# Cumulated pauses

In [ ]:
Dataset['IsNonEvent'] = Dataset['Event'].isna()


In [ ]:
# Calculate the time difference between consecutive rows
Dataset["TimeDifference"] = Dataset.groupby("Fly")["time"].diff()

# For non-event rows, sum these time differences
Dataset["NonEventTime"] = Dataset.loc[Dataset["IsNonEvent"], "TimeDifference"]

# Calculate the cumulative time spent in non-events for each 'Fly' and 'Genotype'
cumulative_time_in_non_events = (
    Dataset.groupby(["Fly", "Nickname", "Simplified region"])["NonEventTime"]
    .sum()
    .reset_index()
)

In [ ]:
h_cumpauses_bp = (
    hv.BoxWhisker(
        data=cumulative_time_in_non_events.reset_index(),
        vdims="NonEventTime",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_cumpauses_sc = (
    hv.Scatter(
        data=cumulative_time_in_non_events.reset_index(),
        vdims="NonEventTime",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_cumpauses = (h_cumpauses_bp * h_cumpauses_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Cumulated time between events (s)",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

# Save the plot
with open(Savepath / "hvplot_cumpauses.pkl", "wb") as f:
    pickle.dump(hvplot_cumpauses, f)

In [ ]:

hvplot_cumpauses

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_cumpauses,
    Savepath / "CumulPauses.html",
)

# Pulls

In [ ]:
# Calculate the difference in 'yball_relative' between the start and end of each event
yball_relative_diff = (
    Dataset.groupby(["Fly", "Event"])["yball_relative"]
    .apply(lambda x: x.iloc[-1] - x.iloc[0])
    .reset_index(name="yball_relative_diff")
)

# Merge yball_relative_diff with Dataset
Dataset = pd.merge(Dataset, yball_relative_diff, on=["Fly", "Event"], how="left")


# Identify whether each event is a 'Push', 'Pull', or 'None'
Dataset["Event_Type"] = np.where(
    Dataset["yball_relative_diff"] > 10,
    "Push",
    np.where(Dataset["yball_relative_diff"] < -10, "Pull", "None"),
)

# Count the number of each event type for each 'Fly' and 'Genotype'
event_type_counts = (
    Dataset.groupby(["Fly", "Nickname", "Event_Type", "Simplified region"]).size().reset_index(name="Count")
)

# Count the number of unique events with event type for each fly and genotype
unique_event_type_counts = (
    event_type_counts.groupby(["Fly", "Nickname", "Simplified region"])["Event_Type"]
    .nunique()
    .reset_index(name="Count")
)

# Filter the DataFrame to include only the 'Pull' events
pull_events = Dataset[Dataset["Event_Type"] == "Pull"]

# Count the number of 'Pull' events for each 'Fly' and 'Genotype'
pull_event_counts = (
    pull_events.groupby(["Fly", "Nickname", "Simplified region"]).size().reset_index(name="Count")
)

# Count the number of unique 'Pull' events for each 'Fly' and 'Genotype'
unique_pull_event_counts = (
    pull_events.groupby(["Fly", "Nickname", "Simplified region"])["Event"]
    .nunique()
    .reset_index(name="Count")
)


In [ ]:
h_pulls_bp = (
    hv.BoxWhisker(
        data=unique_pull_event_counts.reset_index(),
        vdims="Count",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        box_fill_color=None,
        box_line_color="black",
        outlier_fill_color=None,
        outlier_line_color=None,
        framewise=True,
    )
)
h_pulls_sc = (
    hv.Scatter(
        data=unique_pull_event_counts.reset_index(),
        vdims="Count",
        kdims=["Nickname", "Simplified region"],
        color="Nickname",
    )
    .groupby("Simplified region")
    .opts(
        jitter=0.3,
        color="Nickname",
        alpha=0.5,
        size=6,
        cmap="Category10",
        tools=["hover"],
        framewise=True,
    )
)

hvplot_pulls = (h_pulls_bp * h_pulls_sc).opts(
    width=750,
    height=500,
    show_legend=False,
    xlabel="",
    ylabel="Pulling events",
    invert_axes=True,
    show_grid=True,
    fontscale=1.5,
    title="",
    toolbar=None,
)

import pickle
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

# Save the plot
with open(Savepath / "hvplot_pulls.pkl", "wb") as f:
    pickle.dump(hvplot_pulls, f)

In [ ]:

hvplot_pulls

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/Fine_Screening")

hv.save(
    hvplot_pulls,
    Savepath / "Pulling.html",
)

# Events chronology

In [ ]:
# For each 'Fly', 'Event', and 'Genotype', get the timestamp of the first frame
first_frame_timestamps = (
    Dataset.groupby(["Fly", "Event", "Nickname", "Simplified region"])["time"].first().reset_index()
)


In [ ]:

# Plot the data
p_first_frame_timestamps = iqplot.histogram(
    data=first_frame_timestamps,
    q="time",
    cats="Nickname",
    title="Timestamp of first frame of each event",
    frame_width=750,
    frame_height=2000,
)
bokeh.io.show(p_first_frame_timestamps)


In [ ]:

bokeh.io.save(
    p_first_frame_timestamps,
    Savepath / "FirstFrameTimestamps.html",
    # fmt="html",
)

In [ ]:
h_chronology = hv.Histogram(first_frame_timestamps, dimension="time", groupby="Nickname")

h_chronology

In [ ]:
hvplot_chronology

In [ ]:
from bokeh.models import Select, CustomJS
from bokeh.layouts import column, layout
from bokeh.plotting import show, curdoc
import iqplot

# Assuming 'first_frame_timestamps' is your DataFrame
data = first_frame_timestamps

# Get the unique regions
regions = data["Simplified region"].unique()

# Create a dictionary to store the plots for each region
plots = {}

for region in regions:
    # Filter the data for the current region
    data_region = data[data["Simplified region"] == region]

    # Create a histogram for the current region
    p = iqplot.histogram(
        data=data_region,
        q="time",
        cats="Nickname",
        title=f"Timestamp of first frame of each event in {region}",
        frame_width=750,
        frame_height=2000,
    )

    # Store the plot in the dictionary
    plots[region] = p

# Create a dropdown menu with options for each region
select = Select(title="Region", value=regions[0], options=list(regions))

# Create a layout with the dropdown menu and the initial plot
l = layout([[select], [plots[regions[0]]]])

# Define a callback function to update the plot in response to changes in the dropdown menu
callback = CustomJS(
    args=dict(layout=l, plots=plots),
    code="""
    // Get the selected region
    var region = cb_obj.value;
    
    // Get the plot for the selected region
    var plot = plots[region];
    
    // Update the layout
    layout.children[1][0] = plot;
""",
)

# Attach the callback function to the dropdown menu
select.js_on_change("value", callback)

# Display the layout
curdoc().add_root(l)
show(l)

In [ ]:
from bokeh.models import Select, CustomJS
from bokeh.layouts import column, layout
from bokeh.plotting import show, curdoc
from bokeh.models import ColumnDataSource
import iqplot

# Assuming 'first_frame_timestamps' is your DataFrame
data = first_frame_timestamps

# Get the unique regions
regions = data["Simplified region"].unique()

# Create a dictionary to store the plots and data sources for each region
plots = {}
sources = {}

for region in regions:
    # Filter the data for the current region
    data_region = data[data["Simplified region"] == region]

    # Create a histogram for the current region
    p = iqplot.histogram(
        data=data_region,
        q="time",
        cats="Nickname",
        title=f"Timestamp of first frame of each event in {region}",
        frame_width=750,
        frame_height=2000,
    )

    # Store the plot and data source in the dictionaries
    plots[region] = p
    sources[region] = p.renderers[0].data_source

# Create a dropdown menu with options for each region
select = Select(title="Region", value=regions[0], options=list(regions))

# Create a layout with the dropdown menu and the initial plot
l = layout([[select], [plots[regions[0]]]])

# Define a callback function to update the plot in response to changes in the dropdown menu
callback = CustomJS(
    args=dict(layout=l, plots=plots, sources=sources, data=data),
    code="""
    // Get the selected region
    var region = cb_obj.value;
    
    // Get the data source for the selected region
    var source = sources[region];
    
    // Filter the data for the selected region
    var data_region = data[data["Simplified region"] == region];
    
    // Update the data source
    source.data = data_region;
    
    // Update the layout
    layout.children[1][0] = plots[region];
""",
)

# Attach the callback function to the dropdown menu
select.js_on_change("value", callback)

# Display the layout
curdoc().add_root(l)
show(l)